In [99]:
!pip install pymysql
!pip install apyori
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install python-dotenv
!pip install pydot


In [100]:
# Imported Libraries
import pandas as pd
import pymysql
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from apyori import apriori
# Imports for the neural network
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import RFECV, SelectFromModel
from sklearn.linear_model import LogisticRegression
import pickle
# Load the .env file for database credentials
from dotenv import load_dotenv
# Load the .env file to access database credentials
load_dotenv()
####### Connection to Client Database #######
conn = pymysql.connect(host=os.getenv("host"), user=os.getenv("user"), port=int(os.getenv("port")), password=os.getenv("password"), db=os.getenv("dbname"))
# Print connection confirmation
print(conn)

### Getting Customer-users detail - make dataframe

In [101]:
###Customer Personal details
# ! Added Member key
df_user= pd.read_sql("select m.id as member_id, m.membership_number, m.status, m.price_zone_code, m.member_key, s.u_ndis_number, s.u_disabilities, s.u_gender, s.u_date_of_birth, r.SA1, r.SA2, r.SA3,r.SA4  from SNOW_csm_consumer_user s left join  HH_member m  on s.u_ndis_number = m.membership_number left join libe_leapinprod_memberregion r on r.MemberId = s.u_leapin_id where s.u_stage = 'li_managed' and s.u_ndis_number is not null;", con=conn)
                       
df_user.info()
df_user.head(50)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6241 entries, 0 to 6240
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   member_id          6115 non-null   float64
 1   membership_number  6115 non-null   object 
 2   status             6115 non-null   object 
 3   price_zone_code    6115 non-null   object 
 4   member_key         6115 non-null   object 
 5   u_ndis_number      6241 non-null   object 
 6   u_disabilities     6241 non-null   object 
 7   u_gender           6241 non-null   object 
 8   u_date_of_birth    6239 non-null   object 
 9   SA1                6021 non-null   float64
 10  SA2                6021 non-null   float64
 11  SA3                6021 non-null   float64
 12  SA4                6021 non-null   float64
dtypes: float64(5), object(8)
memory usage: 634.0+ KB


,member_id,membership_number,status,price_zone_code,member_key,u_ndis_number,u_disabilities,u_gender,u_date_of_birth,SA1,SA2,SA3,SA4
0,1613.0,916644409,managed,ACT_NSW_QLD_VIC,d74bcd00-d205-11ea-b0d9-53acaecd0b82,916644409,,,1922-11-30,3100614.0,31006.0,30101.0,301.0
1,3132.0,262354568,managed,ACT_NSW_QLD_VIC,5d057910-dce6-11ea-812e-09cdf4382371,262354568,,,2007-04-06,1144614.0,11446.0,12303.0,123.0
2,1786.0,233367363,managed,ACT_NSW_QLD_VIC,222759f0-d212-11ea-8b42-db603b1b9020,233367363,,Female,1954-03-26,0.0,0.0,0.0,0.0
3,2562.0,981297214,managed,ACT_NSW_QLD_VIC,b0a9ca70-d50d-11ea-8224-836b70d82925,981297214,,Female,1978-12-24,3138504.0,31385.0,31401.0,314.0
4,1614.0,631304335,managed,ACT_NSW_QLD_VIC,d75087f0-d205-11ea-b0d9-53acaecd0b82,631304335,,,1995-08-18,2142119.0,21421.0,21701.0,217.0
5,1700.0,67836654,managed,NT_SA_TAS_WA,631efd70-d20b-11ea-ae84-0545f282cdcb,67836654,,Female,1937-01-25,4106012.0,41060.0,40302.0,403.0
6,3803.0,746607176,managed,ACT_NSW_QLD_VIC,f1217e80-e52b-11ea-aa27-0bababe260e0,746607176,I am recovering from a stroke I had in 2,Male,1980-09-28,3131211.0,31312.0,31103.0,311.0
7,1738.0,391263441,managed,ACT_NSW_QLD_VIC,63d81440-d20b-11ea-ae84-0545f282cdcb,391263441,Sciatica,Female,1945-05-23,0.0,0.0,0.0,0.0
8,5162.0,526764245,managed,ACT_NSW_QLD_VIC,e0693f40-1e21-11eb-931b-3f565ba74e99,526764245,,Male,1933-07-11,2120223.0,21202.0,20901.0,209.0
9,1818.0,692294955,managed,ACT_NSW_QLD_VIC,af1f77d0-d216-11ea-bb54-b308798eda15,692294955,,Female,1925-09-09,3123701.0,31237.0,30903.0,309.0


In [102]:
df_user['u_disabilities'].value_counts()

                                                                               4922
Intellectual disability                                                          98
ADHD                                                                             61
Cerebral palsy                                                                   42
Down syndrome                                                                    41
                                                                               ... 
Sensory processing disorder, ADHD, Anxiety, Encopresis, Auditory processing       1
Epilepsy,                                                                         1
Stroke (brain stem), Vision Impaired                                              1
Bipolar disorder, PTSD, Alcohol dependence disorder                               1
ASD level 2  Anxiety  ADHD Sensory Proce                                          1
Name: u_disabilities, Length: 602, dtype: int64

### Getting Services Providers detail - make dataframe

In [103]:
### Service Providers Details:

#df_providers_details = pd.read_sql("select p.id, pa.provider_account_key, p.is_inactive, p.abn, ca.city, ca.country from HH_provider_account pa join HH_provider p on p.id = pa.provider_id join SNOW_customer_account ca on ca.u_abn = p.abn;", con=conn)

#print(df_providers_details.info())

#df_providers_details.head(50)

### Getting Claims and Invoice details - make dataframe

In [104]:
### Getting Claims details: 

df_claims= pd.read_sql("SELECT c.id as claim_id, c.invoice_id, c.state, c.risk_level, c.start_date FROM HH_claim c;", con=conn)
df_claims.info()
df_claims.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968874 entries, 0 to 968873
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   claim_id    968874 non-null  int64 
 1   invoice_id  968874 non-null  int64 
 2   state       968874 non-null  object
 3   risk_level  968874 non-null  object
 4   start_date  968874 non-null  object
dtypes: int64(2), object(3)
memory usage: 37.0+ MB


,claim_id,invoice_id,state,risk_level,start_date
0,1,1,PAID,NO,2020-04-30
1,2,2,PAID,NO,2020-05-01
2,3,3,PAID,NO,2020-04-23
3,4,4,PAID,NO,2020-04-30
4,5,5,PAID,NO,2020-05-07
5,6,6,PAID,NO,2020-05-07
6,7,7,PAID,NO,2020-05-12
7,8,8,PAID,NO,2020-05-14
8,9,9,PAID,NO,2020-05-14
9,10,10,PAID,NO,2020-05-16


In [105]:
###Getting Invoice Details

df_invoices= pd.read_sql("SELECT i.id as invoice_id, i.member_id, i.invoice_total, i.funded_total, i.funded_date FROM HH_invoice i;", con=conn)
df_invoices.info()
df_invoices.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518376 entries, 0 to 518375
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   invoice_id     518376 non-null  int64  
 1   member_id      518376 non-null  int64  
 2   invoice_total  518376 non-null  float64
 3   funded_total   281714 non-null  float64
 4   funded_date    276521 non-null  object 
dtypes: float64(2), int64(2), object(1)
memory usage: 19.8+ MB


,invoice_id,member_id,invoice_total,funded_total,funded_date
0,1,2,160.00,160.00,2020-04-30
1,2,2,160.00,160.00,2020-05-05
2,3,3,80.00,80.00,2020-05-08
3,4,3,80.00,80.00,2020-05-08
4,5,3,80.00,80.00,2020-05-08
5,6,4,214.41,214.41,2020-05-12
6,7,2,130.00,130.00,2020-05-13
7,8,2,130.00,130.00,2020-05-15
8,9,4,428.82,428.82,2020-05-18
9,10,4,214.41,214.41,2020-05-18


In [106]:
### Merging

df_invoice_claim = pd.merge(df_claims, df_invoices, on="invoice_id", how="left")

df_invoice_claim.info()
df_invoice_claim.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 968874 entries, 0 to 968873
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   claim_id       968874 non-null  int64  
 1   invoice_id     968874 non-null  int64  
 2   state          968874 non-null  object 
 3   risk_level     968874 non-null  object 
 4   start_date     968874 non-null  object 
 5   member_id      968874 non-null  int64  
 6   invoice_total  968874 non-null  float64
 7   funded_total   566458 non-null  float64
 8   funded_date    556790 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 73.9+ MB


,claim_id,invoice_id,state,risk_level,start_date,member_id,invoice_total,funded_total,funded_date
0,1,1,PAID,NO,2020-04-30,2,160.0,160.0,2020-04-30
1,2,2,PAID,NO,2020-05-01,2,160.0,160.0,2020-05-05
2,3,3,PAID,NO,2020-04-23,3,80.0,80.0,2020-05-08
3,4,4,PAID,NO,2020-04-30,3,80.0,80.0,2020-05-08
4,5,5,PAID,NO,2020-05-07,3,80.0,80.0,2020-05-08


In [107]:
# Mapping users' disability type:
df = pd.merge(df_invoice_claim, df_user, on="member_id", how="left")
df

,claim_id,invoice_id,state,risk_level,start_date,member_id,invoice_total,funded_total,funded_date,membership_number,...,price_zone_code,member_key,u_ndis_number,u_disabilities,u_gender,u_date_of_birth,SA1,SA2,SA3,SA4
0,1,1,PAID,NO,2020-04-30,2,160.00,160.0,2020-04-30,277486509,...,ACT_NSW_QLD_VIC,ef0ab4c0-891b-11ea-a119-35e8388a5f49,277486509,,Male,1958-04-12,3110117.0,31101.0,30404.0,304.0
1,2,2,PAID,NO,2020-05-01,2,160.00,160.0,2020-05-05,277486509,...,ACT_NSW_QLD_VIC,ef0ab4c0-891b-11ea-a119-35e8388a5f49,277486509,,Male,1958-04-12,3110117.0,31101.0,30404.0,304.0
2,3,3,PAID,NO,2020-04-23,3,80.00,80.0,2020-05-08,815939440,...,ACT_NSW_QLD_VIC,6a535640-8b5c-11ea-9eec-315ed4b1385b,815939440,,Female,1937-10-25,3130913.0,31309.0,31102.0,311.0
3,4,4,PAID,NO,2020-04-30,3,80.00,80.0,2020-05-08,815939440,...,ACT_NSW_QLD_VIC,6a535640-8b5c-11ea-9eec-315ed4b1385b,815939440,,Female,1937-10-25,3130913.0,31309.0,31102.0,311.0
4,5,5,PAID,NO,2020-05-07,3,80.00,80.0,2020-05-08,815939440,...,ACT_NSW_QLD_VIC,6a535640-8b5c-11ea-9eec-315ed4b1385b,815939440,,Female,1937-10-25,3130913.0,31309.0,31102.0,311.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968869,1006095,539315,OPERATOR_AUTHORIZED,NO,2021-04-08,3000,2135.70,NaN,None,265655619,...,ACT_NSW_QLD_VIC,f1a3f970-dc6e-11ea-bd81-13c3c5a9034c,265655619,,Other,1981-04-07,0.0,0.0,0.0,0.0
968870,1006096,539315,OPERATOR_AUTHORIZED,NO,2021-04-10,3000,2135.70,NaN,None,265655619,...,ACT_NSW_QLD_VIC,f1a3f970-dc6e-11ea-bd81-13c3c5a9034c,265655619,,Other,1981-04-07,0.0,0.0,0.0,0.0
968871,1006097,539315,OPERATOR_AUTHORIZED,NO,2021-04-11,3000,2135.70,NaN,None,265655619,...,ACT_NSW_QLD_VIC,f1a3f970-dc6e-11ea-bd81-13c3c5a9034c,265655619,,Other,1981-04-07,0.0,0.0,0.0,0.0
968872,1006098,539316,OPERATOR_AUTHORIZED,NO,2021-04-15,3607,396.57,NaN,None,615053396,...,ACT_NSW_QLD_VIC,df8f8520-ddb7-11ea-a31a-7d25e47a5c0a,615053396,Parkinson's disease,Male,1943-08-12,3136525.0,31365.0,31302.0,313.0


In [108]:
#Groupby member_id 

df1 = df.groupby(["membership_number"]).agg({'invoice_total': 'sum', 'funded_total': 'sum'}).reset_index()

#df_invoice_claim['invoice_total'] = df_invoice_claim.groupby(['member_id'])['invoice_total'].transform('sum')

df1


,membership_number,invoice_total,funded_total
0,100211377,3131.25,3131.25
1,100348708,105389.55,48738.66
2,100633545,99779.60,95339.64
3,100969245,430467.16,386833.71
4,10121655,1260.94,1260.94
...,...,...,...
5723,999135104,13635.22,11058.42
5724,999445370,4736.14,4635.56
5725,999496234,5210.52,4431.51
5726,999694603,17519.26,9040.34


In [109]:
#subtract the invoice_total to fudned_total to see how user get reimbursemente - creat subtraction column

df1['subtraction'] = df1['invoice_total'] - df1['funded_total']
df1

,membership_number,invoice_total,funded_total,subtraction
0,100211377,3131.25,3131.25,0.00
1,100348708,105389.55,48738.66,56650.89
2,100633545,99779.60,95339.64,4439.96
3,100969245,430467.16,386833.71,43633.45
4,10121655,1260.94,1260.94,0.00
...,...,...,...,...
5723,999135104,13635.22,11058.42,2576.80
5724,999445370,4736.14,4635.56,100.58
5725,999496234,5210.52,4431.51,779.01
5726,999694603,17519.26,9040.34,8478.92


In [110]:
### Merging again
df2 = pd.merge(df1, df_user, on="membership_number", how="left")
df2.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5730 entries, 0 to 5729
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   membership_number  5730 non-null   object 
 1   invoice_total      5730 non-null   float64
 2   funded_total       5730 non-null   float64
 3   subtraction        5730 non-null   float64
 4   member_id          5730 non-null   float64
 5   status             5730 non-null   object 
 6   price_zone_code    5730 non-null   object 
 7   member_key         5730 non-null   object 
 8   u_ndis_number      5730 non-null   object 
 9   u_disabilities     5730 non-null   object 
 10  u_gender           5730 non-null   object 
 11  u_date_of_birth    5729 non-null   object 
 12  SA1                5609 non-null   float64
 13  SA2                5609 non-null   float64
 14  SA3                5609 non-null   float64
 15  SA4                5609 non-null   float64
dtypes: float64(8), object(8)

In [111]:
df2.head(20)

,membership_number,invoice_total,funded_total,subtraction,member_id,status,price_zone_code,member_key,u_ndis_number,u_disabilities,u_gender,u_date_of_birth,SA1,SA2,SA3,SA4
0,100211377,3131.25,3131.25,0.00,5793.0,managed,ACT_NSW_QLD_VIC,a8fa0b30-56c4-11eb-a532-d3aaaacb68df,100211377,,Male,1966-02-06,NaN,NaN,NaN,NaN
1,100348708,105389.55,48738.66,56650.89,2700.0,managed,ACT_NSW_QLD_VIC,b33d6da0-d50d-11ea-8224-836b70d82925,100348708,,Female,1966-02-06,0.0,0.0,0.0,0.0
2,100633545,99779.60,95339.64,4439.96,5365.0,managed,ACT_NSW_QLD_VIC,02e01760-2ac0-11eb-96d1-5d2a1e5a1f4e,100633545,,Female,1991-02-06,3125124.0,31251.0,30907.0,309.0
3,100969245,430467.16,386833.71,43633.45,3267.0,managed,ACT_NSW_QLD_VIC,d3d41ed0-dcef-11ea-a5ac-61abce895864,100969245,,Female,1967-02-06,3136525.0,31365.0,31302.0,313.0
4,10121655,1260.94,1260.94,0.00,5642.0,managed,ACT_NSW_QLD_VIC,fa601d30-4336-11eb-914b-837581d0325d,10121655,,Male,1981-01-04,0.0,0.0,0.0,0.0
5,101279515,4810.00,4180.00,630.00,1092.0,managed,ACT_NSW_QLD_VIC,f38d5e50-c7f3-11ea-baa6-ebfe92aca975,101279515,,Female,1990-02-06,0.0,0.0,0.0,0.0
6,101391416,107771.99,94603.44,13168.55,1042.0,managed,ACT_NSW_QLD_VIC,f28479d0-c7f3-11ea-baa6-ebfe92aca975,101391416,Intellectual disability,Female,1967-02-06,3124942.0,31249.0,30906.0,309.0
7,101477884,28537.95,28291.27,246.68,3712.0,managed,NT_SA_TAS_WA,20a70250-e2b7-11ea-8713-199aa8c5c32a,101477884,Anxiety,Male,1992-02-06,0.0,0.0,0.0,0.0
8,101503314,103055.72,102695.78,359.94,5497.0,managed,NT_SA_TAS_WA,3b4191a0-352f-11eb-9292-8b2fd1c3e301,101503314,,Female,1941-02-06,5104305.0,51043.0,50302.0,503.0
9,101528743,17048064.52,5614195.49,11433869.03,3611.0,managed,ACT_NSW_QLD_VIC,dfa55710-ddb7-11ea-a31a-7d25e47a5c0a,101528743,,,1967-02-06,0.0,0.0,0.0,0.0


In [112]:
#Select interested columns
cols_of_interest = {'membership_number', 'invoice_total', 'funded_total', 'subtraction', 'status', 'price_zone_code', 
                   'u_disabilities', 'u_gender'}
df2 = df2[cols_of_interest]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5730 entries, 0 to 5729
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   funded_total       5730 non-null   float64
 1   subtraction        5730 non-null   float64
 2   membership_number  5730 non-null   object 
 3   status             5730 non-null   object 
 4   u_gender           5730 non-null   object 
 5   u_disabilities     5730 non-null   object 
 6   invoice_total      5730 non-null   float64
 7   price_zone_code    5730 non-null   object 
dtypes: float64(3), object(5)
memory usage: 402.9+ KB


In [113]:
#Re-arrange that mess

df2 = df2[['membership_number', 'invoice_total', 'funded_total', 'subtraction',   
                   'u_disabilities', 'u_gender','price_zone_code','status']]

df2.head(20)

,membership_number,invoice_total,funded_total,subtraction,u_disabilities,u_gender,price_zone_code,status
0,100211377,3131.25,3131.25,0.00,,Male,ACT_NSW_QLD_VIC,managed
1,100348708,105389.55,48738.66,56650.89,,Female,ACT_NSW_QLD_VIC,managed
2,100633545,99779.60,95339.64,4439.96,,Female,ACT_NSW_QLD_VIC,managed
3,100969245,430467.16,386833.71,43633.45,,Female,ACT_NSW_QLD_VIC,managed
4,10121655,1260.94,1260.94,0.00,,Male,ACT_NSW_QLD_VIC,managed
5,101279515,4810.00,4180.00,630.00,,Female,ACT_NSW_QLD_VIC,managed
6,101391416,107771.99,94603.44,13168.55,Intellectual disability,Female,ACT_NSW_QLD_VIC,managed
7,101477884,28537.95,28291.27,246.68,Anxiety,Male,NT_SA_TAS_WA,managed
8,101503314,103055.72,102695.78,359.94,,Female,NT_SA_TAS_WA,managed
9,101528743,17048064.52,5614195.49,11433869.03,,,ACT_NSW_QLD_VIC,managed


In [114]:
#Checking u_dis

df2['u_disabilities'].value_counts()

                                                                                                                   4493
Intellectual disability                                                                                              93
ADHD                                                                                                                 59
Down syndrome                                                                                                        40
Cerebral palsy                                                                                                       39
                                                                                                                   ... 
Relapsing remitting , Multiple Sclerosis                                                                              1
Autism with anxiety                                                                                                   1
Anxiety, Neurogenic Bladder & Bowel, ADH

In [115]:
#df2['u_disabilities'].mask(df2['u_disabilities'] == ' ', "others", inplace = False)
df2['u_disabilities'] = df2['u_disabilities'].replace([''],'others')

df2['u_disabilities'].value_counts()

others                                                                                                             4493
Intellectual disability                                                                                              93
ADHD                                                                                                                 59
Down syndrome                                                                                                        40
Cerebral palsy                                                                                                       39
                                                                                                                   ... 
Anxiety, Suicidal, Inflamed Bersers In Shoulders, Social Seclusion , Paraplegia, Depression                           1
Relapsing remitting , Multiple Sclerosis                                                                              1
Autism with anxiety                     

In [116]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5730 entries, 0 to 5729
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   membership_number  5730 non-null   object 
 1   invoice_total      5730 non-null   float64
 2   funded_total       5730 non-null   float64
 3   subtraction        5730 non-null   float64
 4   u_disabilities     5730 non-null   object 
 5   u_gender           5730 non-null   object 
 6   price_zone_code    5730 non-null   object 
 7   status             5730 non-null   object 
dtypes: float64(3), object(5)
memory usage: 402.9+ KB


In [117]:
sub_0 = df2[df2['subtraction']> 1000]
#sub_0

sub_1 = df2[df2['subtraction']> 10000]
sub_1

#Set value for rows matching condition

#df2[['subtraction'] > 1000] == 'High'
#df2


,membership_number,invoice_total,funded_total,subtraction,u_disabilities,u_gender,price_zone_code,status
1,100348708,105389.55,48738.66,56650.89,others,Female,ACT_NSW_QLD_VIC,managed
3,100969245,430467.16,386833.71,43633.45,others,Female,ACT_NSW_QLD_VIC,managed
6,101391416,107771.99,94603.44,13168.55,Intellectual disability,Female,ACT_NSW_QLD_VIC,managed
9,101528743,17048064.52,5614195.49,11433869.03,others,,ACT_NSW_QLD_VIC,managed
11,101701683,61064.03,43086.19,17977.84,others,Female,NT_SA_TAS_WA,managed
...,...,...,...,...,...,...,...,...
5709,99728172,808981.78,471210.21,337771.57,others,Male,ACT_NSW_QLD_VIC,managed
5718,998565427,20985.75,8374.39,12611.36,others,,ACT_NSW_QLD_VIC,managed
5720,998763796,51599.64,32850.29,18749.35,others,,ACT_NSW_QLD_VIC,managed
5723,998987595,26453.97,13622.03,12831.94,"ADHD,",Male,NT_SA_TAS_WA,managed


In [118]:
# Modifications to D2
cleaned_df2 = df2
# gender		: Add "other" for blank or null values --> Male/Female/Other/Unknown
cleaned_df2["u_gender"] = cleaned_df2["u_gender"].replace([""],"Unknown")
# status		: drop
cleaned_df2 = cleaned_df2.drop(columns=["status"])
# price_code	: drop, replace with SA4 and its information
cleaned_df2 = cleaned_df2.drop(columns=["price_zone_code"])
df_user_min = df_user[["membership_number", "SA4"]]
# Remove rows with blank SA1 - SA4 values
blankIndices = df_user_min[df_user_min["SA4"] == 0.0].index
df_user_min = df_user_min.drop(axis=0, labels=blankIndices)
# Drop rows with null SA1-4 values 
df_user_min = df_user_min.dropna()
# Drop rows with duplicate membership_number values 
df_user_min = df_user_min.drop_duplicates()
# Obtain geographical locations using SA4
df_sa4 = pd.read_csv("./TableDump/SA4_2016.csv")
# Extract relevant columns
df_sa4 = df_sa4[["SA4_CODE_2016", "SA4_NAME_2016", "GCCSA_NAME_2016", "STATE_NAME_2016"]]
# Rename columns to match df2
name_mapping = {
    "SA4_CODE_2016": "SA4",
    "SA4_NAME_2016": "SA4_NAME",
    "GCCSA_NAME_2016": "GCCSA_NAME",
    "STATE_NAME_2016": "STATE_NAME"
}
df_sa4 = df_sa4.rename(columns=name_mapping)
# Merge SA4 information with the user DF
df_user_min = pd.merge(df_user_min, df_sa4, on="SA4", how="left")
# Merge the final user info with DF2
cleaned_df2 = pd.merge(cleaned_df2, df_user_min, on="membership_number", how="left")
# Add Member_key to deal with invoice calculations
cleaned_df2 = pd.merge(cleaned_df2, df_user[["membership_number", "member_key"]], on="membership_number", how="left")
# Drop duplicated member keys
cleaned_df2 = cleaned_df2.drop_duplicates()
# Display Summary information for cleaned_df2
cleaned_df2.info()
cleaned_df2.head(100)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5730 entries, 0 to 5733
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   membership_number  5730 non-null   object 
 1   invoice_total      5730 non-null   float64
 2   funded_total       5730 non-null   float64
 3   subtraction        5730 non-null   float64
 4   u_disabilities     5730 non-null   object 
 5   u_gender           5730 non-null   object 
 6   SA4                3521 non-null   float64
 7   SA4_NAME           3521 non-null   object 
 8   GCCSA_NAME         3521 non-null   object 
 9   STATE_NAME         3521 non-null   object 
 10  member_key         5730 non-null   object 
dtypes: float64(4), object(7)
memory usage: 537.2+ KB


,membership_number,invoice_total,funded_total,subtraction,u_disabilities,u_gender,SA4,SA4_NAME,GCCSA_NAME,STATE_NAME,member_key
0,100211377,3131.25,3131.25,0.00,others,Male,NaN,NaN,NaN,NaN,a8fa0b30-56c4-11eb-a532-d3aaaacb68df
1,100348708,105389.55,48738.66,56650.89,others,Female,NaN,NaN,NaN,NaN,b33d6da0-d50d-11ea-8224-836b70d82925
2,100633545,99779.60,95339.64,4439.96,others,Female,309.0,Gold Coast,Rest of Qld,Queensland,02e01760-2ac0-11eb-96d1-5d2a1e5a1f4e
3,100969245,430467.16,386833.71,43633.45,others,Female,313.0,Moreton Bay - North,Greater Brisbane,Queensland,d3d41ed0-dcef-11ea-a5ac-61abce895864
4,10121655,1260.94,1260.94,0.00,others,Male,NaN,NaN,NaN,NaN,fa601d30-4336-11eb-914b-837581d0325d
...,...,...,...,...,...,...,...,...,...,...,...
95,113842839,35468.38,18694.88,16773.50,others,Female,NaN,NaN,NaN,NaN,3585e7c0-e741-11ea-804b-97facb641bb5
96,11388161,2918.66,2918.66,0.00,others,Female,214.0,Mornington Peninsula,Greater Melbourne,Victoria,bb1a9220-d7b3-11ea-a51d-3f9e5f9119be
97,114127679,18187.93,18187.95,-0.02,others,Female,310.0,Ipswich,Greater Brisbane,Queensland,8457dc40-5948-11eb-90e2-394d2efb95d8
98,114463375,4583.05,3661.09,921.96,others,Male,313.0,Moreton Bay - North,Greater Brisbane,Queensland,fb69cda0-d23b-11ea-a13a-d188ddbdc773


In [119]:
# days_funded: funded_date - start_date from , to get the duration of reimbursement
df_plan = pd.read_sql("select p.plan_key, p.member_key, p.status, p.start_date, p.end_date, pb.item_category_level2_key, pb.allocation, pb.remaining from HH_plan p join HH_plan_budget pb on p.plan_key = pb.plan_key where p.status = 'COMPLETED'", con=conn)
# Convert start and end_date to datetime
df_plan["start_date"] = pd.to_datetime(df_plan["start_date"], format="%Y-%m-%d")
df_plan["end_date"] = pd.to_datetime(df_plan["end_date"], format="%Y-%m-%d")
# Summary info for df_plan
df_plan.info()
df_plan.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46587 entries, 0 to 46586
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   plan_key                  46587 non-null  object        
 1   member_key                46587 non-null  object        
 2   status                    46587 non-null  object        
 3   start_date                46587 non-null  datetime64[ns]
 4   end_date                  46587 non-null  datetime64[ns]
 5   item_category_level2_key  45825 non-null  object        
 6   allocation                46587 non-null  float64       
 7   remaining                 46587 non-null  float64       
dtypes: datetime64[ns](2), float64(2), object(4)
memory usage: 2.8+ MB


,plan_key,member_key,status,start_date,end_date,item_category_level2_key,allocation,remaining
0,dd735ad0-05eb-42ce-91ba-36e1e032a67a,472c17f0-9409-11ea-ba24-1f1ea52ce5bc,COMPLETED,2019-09-11,2020-08-17,7d0af57b-f597-11e9-bfb4-022d4762bb3c,6972.41,6972.41
1,dd735ad0-05eb-42ce-91ba-36e1e032a67a,472c17f0-9409-11ea-ba24-1f1ea52ce5bc,COMPLETED,2019-09-11,2020-08-17,7d0af71b-f597-11e9-bfb4-022d4762bb3c,0.05,0.05
2,dd735ad0-05eb-42ce-91ba-36e1e032a67a,472c17f0-9409-11ea-ba24-1f1ea52ce5bc,COMPLETED,2019-09-11,2020-08-17,7d0af5b1-f597-11e9-bfb4-022d4762bb3c,0.05,0.05
3,dd735ad0-05eb-42ce-91ba-36e1e032a67a,472c17f0-9409-11ea-ba24-1f1ea52ce5bc,COMPLETED,2019-09-11,2020-08-17,7d0af52b-f597-11e9-bfb4-022d4762bb3c,0.05,0.05
4,dd735ad0-05eb-42ce-91ba-36e1e032a67a,472c17f0-9409-11ea-ba24-1f1ea52ce5bc,COMPLETED,2019-09-11,2020-08-17,7d0af53b-f597-11e9-bfb4-022d4762bb3c,0.05,0.05
...,...,...,...,...,...,...,...,...
95,7e7decf0-bc32-11ea-bb26-9bde36a91794,7d1a6550-bc32-11ea-840d-791c3ea82cc6,COMPLETED,2020-01-25,2021-01-26,7d0af5b1-f597-11e9-bfb4-022d4762bb3c,2000.00,29.05
96,7e7decf0-bc32-11ea-bb26-9bde36a91794,7d1a6550-bc32-11ea-840d-791c3ea82cc6,COMPLETED,2020-01-25,2021-01-26,7d0af53b-f597-11e9-bfb4-022d4762bb3c,200.00,1.97
97,7e7decf0-bc32-11ea-bb26-9bde36a91794,7d1a6550-bc32-11ea-840d-791c3ea82cc6,COMPLETED,2020-01-25,2021-01-26,7d0af57b-f597-11e9-bfb4-022d4762bb3c,20105.52,10808.94
98,da0f67bc-7ec9-4a55-b331-169e175bcb3c,75f76ea0-bc35-11ea-a004-cd564a146b2a,COMPLETED,2020-06-24,2020-07-28,7d0af57b-f597-11e9-bfb4-022d4762bb3c,2133.90,2133.90


In [120]:
df_plan2 = df_plan
# Convert start and end_date to datetime
df_plan2["start_date"] = pd.to_datetime(df_plan2["start_date"], format="%Y-%m-%d")
df_plan2["end_date"] = pd.to_datetime(df_plan2["end_date"], format="%Y-%m-%d")
# Sum up all based on plan_key
df_grouped_plan2 = df_plan2.groupby(["plan_key"]).agg({"allocation": "sum", "remaining": "sum", "member_key": "first", "start_date": "first", "end_date": "first"}).reset_index()
# Sort grouped df by start and end dates in descending order
df_grouped_plan2 = df_grouped_plan2.sort_values(["start_date", "end_date"], ascending=[False, False])
# Eliminate duplicate member keys by dropping all rows but the most recent ones
df_grouped_plan2 = df_grouped_plan2.groupby(["member_key"]).agg({"start_date": "first", "end_date": "first", "plan_key": "first", "allocation": "first", "remaining": "first"}).reset_index()
# Summary information for df_plan2
df_grouped_plan2.info()
df_grouped_plan2.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3781 entries, 0 to 3780
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   member_key  3781 non-null   object        
 1   start_date  3781 non-null   datetime64[ns]
 2   end_date    3781 non-null   datetime64[ns]
 3   plan_key    3781 non-null   object        
 4   allocation  3781 non-null   float64       
 5   remaining   3781 non-null   float64       
dtypes: datetime64[ns](2), float64(2), object(2)
memory usage: 177.4+ KB


,member_key,start_date,end_date,plan_key,allocation,remaining
0,00117a50-cbc6-11ea-805a-75fc51d39a77,2020-04-06,2021-04-11,8aa26238-d922-4963-bcaf-427d24a610bc,64524.00,48019.14
1,007c1090-d6ca-11ea-bb8a-a73bf14c75f9,2020-08-14,2020-08-17,8910b6c2-643b-421e-9da6-8f2fea8f3e9a,13165.40,13165.40
2,009cfe40-c630-11ea-90f7-3faa82d32fb9,2019-10-15,2020-09-23,2b2ed78b-f926-438a-baf9-01144eabbc5e,96278.43,88172.54
3,0250ad20-cc90-11ea-8ccf-0d690c771492,2020-07-20,2020-11-26,10cb6fe5-acd4-4367-a683-5b31dda02c84,253879.36,167869.14
4,02553f70-d861-11ea-b574-9762bc5dde4a,2020-07-27,2020-09-24,524b880c-dbf9-46fa-955f-e1e4e89acdab,112366.90,110312.30
...,...,...,...,...,...,...
95,0533b850-e588-11ea-b453-3de7750d4ba6,2019-08-20,2020-03-09,08c3e3f0-e588-11ea-be19-95ba49e7f45d,13558.82,13558.82
96,0538e870-e588-11ea-b453-3de7750d4ba6,2019-12-24,2020-04-20,09c8a9c0-e588-11ea-be19-95ba49e7f45d,30751.38,25415.72
97,053e1890-e588-11ea-b453-3de7750d4ba6,2020-06-15,2021-01-12,078e7130-e588-11ea-be19-95ba49e7f45d,47136.72,43185.58
98,054348b0-e588-11ea-b453-3de7750d4ba6,2020-08-28,2021-01-10,848e46bf-f1a1-4bbf-91b8-f7f65473a724,886368.63,235018.89


In [153]:
# Merge plan information with cleaned_df2
temp_df2 = pd.merge(cleaned_df2, df_grouped_plan2, on="member_key", how="left")
# Drop members without completed plans
temp_df2 = temp_df2.dropna(subset=["plan_key"])
# Recalculate the "subtraction" column
# TODO: Find the plan progress rate: 100% for completion
# Get current date. check if diff between current day and start > end and start
# currentDate = pd.datetime.now().date()
# if yes, 100% completion
# if no, in progress. 
# ratio: (current_date - start_date) / (end_date - start_date)
# temp_df2["plan_duration_elapsed"] = temp_df2["start_date"] / temp_df2["allocation"]
# TODO: compare progress to allocated and spent. if the ratios are comparable, then alright. if skewed, problematic
# ratio: (spent) / (allocated)
temp_df2["spending_ratio"] = temp_df2["remaining"] / temp_df2["allocation"]
# If different ratios, under/overspend. 
# TODO: CHECK AND MODIFY VALUES HERE
under_spend_thres = 0.75
par_spend_thres = 1.0
temp_df2["under_spent"] = temp_df2["spending_ratio"] <= under_spend_thres
temp_df2["over_spent"] = temp_df2["spending_ratio"] > par_spend_thres
temp_df2["par_spent"] = temp_df2["spending_ratio"] == par_spend_thres
# Drop outdated columns
temp_df2 = temp_df2.drop(columns=["invoice_total", "funded_total"])
# Display Summary information for cleaned_df2
temp_df2.info()
temp_df2.head(100)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3390 entries, 1 to 5728
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   membership_number  3390 non-null   object        
 1   subtraction        3390 non-null   float64       
 2   u_disabilities     3390 non-null   object        
 3   u_gender           3390 non-null   object        
 4   SA4                1910 non-null   float64       
 5   SA4_NAME           1910 non-null   object        
 6   GCCSA_NAME         1910 non-null   object        
 7   STATE_NAME         1910 non-null   object        
 8   member_key         3390 non-null   object        
 9   start_date         3390 non-null   datetime64[ns]
 10  end_date           3390 non-null   datetime64[ns]
 11  plan_key           3390 non-null   object        
 12  allocation         3390 non-null   float64       
 13  remaining          3390 non-null   float64       
 14  spending

,membership_number,subtraction,u_disabilities,u_gender,SA4,SA4_NAME,GCCSA_NAME,STATE_NAME,member_key,start_date,end_date,plan_key,allocation,remaining,spending_ratio,under_spent,over_spent,par_spent
1,100348708,56650.89,others,Female,NaN,NaN,NaN,NaN,b33d6da0-d50d-11ea-8224-836b70d82925,2020-01-14,2021-01-20,be26b3c0-d50d-11ea-a941-f79507f5b8c8,124999.40,51830.29,0.414644,True,False,False
3,100969245,43633.45,others,Female,313.0,Moreton Bay - North,Greater Brisbane,Queensland,d3d41ed0-dcef-11ea-a5ac-61abce895864,2019-03-12,2020-05-24,dcc78860-dcef-11ea-973a-cfc6cbcf0bd6,57601.24,56214.83,0.975931,False,False,False
5,101279515,630.00,others,Female,NaN,NaN,NaN,NaN,f38d5e50-c7f3-11ea-baa6-ebfe92aca975,2020-02-13,2021-02-09,ffa846a0-c7f3-11ea-b247-07c902e1140e,14054.04,9438.47,0.671584,True,False,False
6,101391416,13168.55,Intellectual disability,Female,309.0,Gold Coast,Rest of Qld,Queensland,f28479d0-c7f3-11ea-baa6-ebfe92aca975,2019-11-29,2020-11-15,00b1ee70-c7f4-11ea-b247-07c902e1140e,220504.96,64026.02,0.290361,True,False,False
9,101528743,11433869.03,others,Unknown,NaN,NaN,NaN,NaN,dfa55710-ddb7-11ea-a31a-7d25e47a5c0a,2021-03-12,2021-03-15,1cf38ad9-1cb1-4025-a0aa-e3e7736d4f68,224608.59,224608.59,1.000000,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,12344398,19192.28,Down syndrome,Female,217.0,Warrnambool and South West,Rest of Vic.,Victoria,c2eb5e70-d6d3-11ea-9cd1-f194648955a5,2020-03-21,2021-03-24,c7d092c0-d6d3-11ea-b1ee-87504086bf09,393071.73,278667.39,0.708948,True,False,False
157,123476419,142631.00,others,Female,NaN,NaN,NaN,NaN,de887560-ddb7-11ea-a31a-7d25e47a5c0a,2019-11-13,2020-12-16,e7c87850-ddb7-11ea-8b25-0fa931eaa230,416043.34,230512.00,0.554058,True,False,False
160,123786685,42643.81,others,Female,310.0,Ipswich,Greater Brisbane,Queensland,f31213d0-c7f3-11ea-baa6-ebfe92aca975,2020-01-24,2021-03-14,01f36f20-c7f4-11ea-b247-07c902e1140e,645045.88,413846.93,0.641578,True,False,False
162,123985054,455.00,others,Male,305.0,Brisbane Inner City,Greater Brisbane,Queensland,dc4f9cb0-c179-11ea-8a11-397bdc21f5a9,2020-04-08,2021-04-08,e1eb01a0-c179-11ea-8daf-e7e742291ebd,1085.68,175.68,0.161816,True,False,False


In [154]:
# One-Hot Encoding of cleaned_df2
one_hot_df2 = temp_df2.set_index("membership_number")
# One-hot encoding of Gender
one_hot_gender = pd.get_dummies(one_hot_df2["u_gender"], prefix="gender")
# Merge with the cleaned DF2
one_hot_df2 = pd.merge(one_hot_df2, one_hot_gender, on="membership_number", how="left")
# One-hot encoding of GCCSA
# Compile list of Greater regions
greaterRegions = one_hot_df2[one_hot_df2["GCCSA_NAME"].str.contains("Greater", na=False)]["GCCSA_NAME"].value_counts().index.to_list()
# Compile list of "Rest of..." regions
restOfRegions = one_hot_df2[one_hot_df2["GCCSA_NAME"].str.contains("Rest of", na=False)]["GCCSA_NAME"].value_counts().index.to_list()
# Compile list of "Capital" regions
capitalRegions = one_hot_df2[one_hot_df2["GCCSA_NAME"].str.contains("Capital", na=False)]["GCCSA_NAME"].value_counts().index.to_list()
# Replace "Greater" values with Urban
for region in greaterRegions:
    one_hot_df2 = one_hot_df2.replace([region], "Urban")
# Replace "Rest of..." values with Rural
for region in restOfRegions:
    one_hot_df2 = one_hot_df2.replace([region], "Rural")
# Replace "Capital" regions with Urban
for region in capitalRegions:
    one_hot_df2 = one_hot_df2.replace([region], "Urban")
# Replace NaN values with "Unknown"
one_hot_df2["GCCSA_NAME"] = one_hot_df2["GCCSA_NAME"].replace(np.nan, "Unknown")
# Perform One-hot Encoding on GCCSA_NAME
one_hot_region = pd.get_dummies(one_hot_df2["GCCSA_NAME"], prefix="GCCSA")
one_hot_df2 = pd.merge(one_hot_df2, one_hot_region, on="membership_number", how="left")
# Convert any UINT8 columns to Bool
column_names = one_hot_df2.select_dtypes(include=[np.uint8]).columns
one_hot_df2[column_names] = one_hot_df2[column_names].astype(bool)
# Drop unnecessary columns
one_hot_df2 = one_hot_df2.drop(columns=["SA4", "SA4_NAME", "GCCSA_NAME", "STATE_NAME", "u_gender"])
# Display Summary information for one_hot_df2
one_hot_df2.info()
one_hot_df2.head(100)

<class 'pandas.core.frame.DataFrame'>
Index: 3404 entries, 100348708 to 999694603
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   subtraction     3404 non-null   float64       
 1   u_disabilities  3404 non-null   object        
 2   member_key      3404 non-null   object        
 3   start_date      3404 non-null   datetime64[ns]
 4   end_date        3404 non-null   datetime64[ns]
 5   plan_key        3404 non-null   object        
 6   allocation      3404 non-null   float64       
 7   remaining       3404 non-null   float64       
 8   spending_ratio  3404 non-null   float64       
 9   under_spent     3404 non-null   bool          
 10  over_spent      3404 non-null   bool          
 11  par_spent       3404 non-null   bool          
 12  gender_Female   3404 non-null   bool          
 13  gender_Male     3404 non-null   bool          
 14  gender_Other    3404 non-null   bool          
 

,subtraction,u_disabilities,member_key,start_date,end_date,plan_key,allocation,remaining,spending_ratio,under_spent,over_spent,par_spent,gender_Female,gender_Male,gender_Other,gender_Unknown,GCCSA_Rural,GCCSA_Unknown,GCCSA_Urban
membership_number,,,,,,,,,,,,,,,,,,,
100348708,56650.89,others,b33d6da0-d50d-11ea-8224-836b70d82925,2020-01-14,2021-01-20,be26b3c0-d50d-11ea-a941-f79507f5b8c8,124999.40,51830.29,0.414644,True,False,False,True,False,False,False,False,True,False
100969245,43633.45,others,d3d41ed0-dcef-11ea-a5ac-61abce895864,2019-03-12,2020-05-24,dcc78860-dcef-11ea-973a-cfc6cbcf0bd6,57601.24,56214.83,0.975931,False,False,False,True,False,False,False,False,False,True
101279515,630.00,others,f38d5e50-c7f3-11ea-baa6-ebfe92aca975,2020-02-13,2021-02-09,ffa846a0-c7f3-11ea-b247-07c902e1140e,14054.04,9438.47,0.671584,True,False,False,True,False,False,False,False,True,False
101391416,13168.55,Intellectual disability,f28479d0-c7f3-11ea-baa6-ebfe92aca975,2019-11-29,2020-11-15,00b1ee70-c7f4-11ea-b247-07c902e1140e,220504.96,64026.02,0.290361,True,False,False,True,False,False,False,True,False,False
101528743,11433869.03,others,dfa55710-ddb7-11ea-a31a-7d25e47a5c0a,2021-03-12,2021-03-15,1cf38ad9-1cb1-4025-a0aa-e3e7736d4f68,224608.59,224608.59,1.000000,False,False,True,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12344398,19192.28,Down syndrome,c2eb5e70-d6d3-11ea-9cd1-f194648955a5,2020-03-21,2021-03-24,c7d092c0-d6d3-11ea-b1ee-87504086bf09,393071.73,278667.39,0.708948,True,False,False,True,False,False,False,True,False,False
123476419,142631.00,others,de887560-ddb7-11ea-a31a-7d25e47a5c0a,2019-11-13,2020-12-16,e7c87850-ddb7-11ea-8b25-0fa931eaa230,416043.34,230512.00,0.554058,True,False,False,True,False,False,False,False,True,False
123786685,42643.81,others,f31213d0-c7f3-11ea-baa6-ebfe92aca975,2020-01-24,2021-03-14,01f36f20-c7f4-11ea-b247-07c902e1140e,645045.88,413846.93,0.641578,True,False,False,True,False,False,False,False,False,True


## Neural Network on DF2



In [159]:
final_df2 = one_hot_df2
# Drop columns with unique values. FOR NOW, no DISABILITIES
final_df2 = final_df2.drop(columns=["plan_key", "member_key", "subtraction", "u_disabilities", "start_date", "end_date"])
# Summary information for final_df2
final_df2.info()
final_df2.head(100)
# Specify the target column for analysis
target_column = "under_spent"
# Create the input df
input_df2 = final_df2.drop([target_column], axis=1)
# Create a target df
target_df2 = final_df2[target_column]
# Set random state
random_state = 10
# Set test size
test_size = 0.3
# Nump-ify input_df2
input_df2_mat = input_df2.to_numpy()
# Split training and test data
input_df2_train, input_df2_test, target_df2_train, target_df2_test = train_test_split(input_df2_mat, target_df2, test_size=test_size, stratify=target_df2, random_state=random_state)
# Get standard scaler
scaler = StandardScaler()
# Transform training and test data
input_df2_train = scaler.fit_transform(input_df2_train, target_df2_train)
input_df2_test = scaler.transform(input_df2_test)
# Generate a prediction
model_1 = MLPClassifier(random_state=random_state)
model_1.fit(input_df2_train, target_df2_train)
target_prediction = model_1.predict(input_df2_test)
# Summary Information
print("Classification Report: \n", classification_report(target_df2_test, target_prediction))
print("Train Accuracy: ", model_1.score(input_df2_train, target_df2_train))
print("Test Accuracy: ", model_1.score(input_df2_test, target_df2_test))
print("Default Model Characteristics: ", model_1)



<class 'pandas.core.frame.DataFrame'>
Index: 3404 entries, 100348708 to 999694603
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   allocation      3404 non-null   float64
 1   remaining       3404 non-null   float64
 2   spending_ratio  3404 non-null   float64
 3   under_spent     3404 non-null   bool   
 4   over_spent      3404 non-null   bool   
 5   par_spent       3404 non-null   bool   
 6   gender_Female   3404 non-null   bool   
 7   gender_Male     3404 non-null   bool   
 8   gender_Other    3404 non-null   bool   
 9   gender_Unknown  3404 non-null   bool   
 10  GCCSA_Rural     3404 non-null   bool   
 11  GCCSA_Unknown   3404 non-null   bool   
 12  GCCSA_Urban     3404 non-null   bool   
dtypes: bool(10), float64(3)
memory usage: 268.7+ KB
Classification Report: 
               precision    recall  f1-score   support

       False       0.99      1.00      0.99       485
        True       1.00

In [161]:
# Improved Model - Neural Network tuned with GridSearchCV
hiddenLayerSizes = [(2,), (3, ), (4, ), (5, ), (6, ), (7, )]

alpha = [0.01, 0.001, 0.0001, 0.00001]

params = {'hidden_layer_sizes': hiddenLayerSizes, 'alpha': alpha}

model_2 = GridSearchCV(param_grid=params, estimator=MLPClassifier(random_state=random_state), cv=10, n_jobs=-1)

model_2.fit(input_df2_train, target_df2_train)

target_prediction = model_2.predict(input_df2_test)
# Summary Information
print("Classification Report: \n", classification_report(target_df2_test, target_prediction))
print("Train Accuracy: ", model_2.score(input_df2_train, target_df2_train))
print("Test Accuracy: ", model_2.score(input_df2_test, target_df2_test))
print("Tuned Model Characteristics: ", model_2)
print("Best Parameters: \n", model_2.best_params_)

Classification Report: 
               precision    recall  f1-score   support

       False       0.99      1.00      0.99       485
        True       1.00      0.99      0.99       537

    accuracy                           0.99      1022
   macro avg       0.99      0.99      0.99      1022
weighted avg       0.99      0.99      0.99      1022

Train Accuracy:  0.9966414777497901
Test Accuracy:  0.9921722113502935
Tuned Model Characteristics:  GridSearchCV(cv=10, estimator=MLPClassifier(random_state=10), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.001, 0.0001, 1e-05],
                         'hidden_layer_sizes': [(2,), (3,), (4,), (5,), (6,),
                                                (7,)]})
Best Parameters: 
 {'alpha': 0.01, 'hidden_layer_sizes': (3,)}
C:\Users\ihand\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the opti

In [50]:
'''
### All Claim details (took 2 minutes to execute)

df_all_claims = pd.read_sql("select c.id as claim_id, invoice_id as invoiceId, c.item_category_level3_id from HH_claim c", con=conn)

#print(df_all_claims.info())
#print(df_all_claims)
### All item_categories details. Took few seconds for execution.

##  
df_all_ndis_service_cat = pd.read_sql("select item_category_level3_id, ndis.registration_group from hedgehog_ndis_service_item_ref ndis", con=conn)
#print(df_all_ndis_service_cat.info())
#print(df_all_ndis_service_cat)

##Combine df_all_claims and df_all_ndis_service_cat
df_claimsWithProvider_details = pd.merge(df_all_claims, df_all_ndis_service_cat, on="item_category_level3_id", how= 'inner')
df_claimsWithProvider_details= df_claimsWithProvider_details.drop(columns=['item_category_level3_id'])
#df_claimsWithProvider_details = pd.read_sql("select c.id, c.invoice_id as invoiceId, n.registration_group from HH_claim c left join hedgehog_ndis_service_item_ref n on c.item_category_level3_id = n.item_category_level3_id;", con=conn)
#print(df_claimsWithProvider_details.info())
#print(df_claimsWithProvider_details)

### Provider Services details (Provider account linked to invoice and HH_provider) took 1 minute to execute
##  
df_providersWithInvoice_details = pd.read_sql("select i.id as invoiceId, i.provider_account_id, (select p.abn from HH_provider p where pa.provider_id= p.id) as provider_abn from HH_invoice i left join HH_provider_account pa on i.provider_account_id = pa.id;", con=conn)
#print(df_providersWithInvoice_details.info())
#print(df_providersWithInvoice_details)

##Combine df_claim_providers_details and df_providersWithInvoice_details
df_claims_provider_details= pd.merge(df_claimsWithProvider_details, df_providersWithInvoice_details, on="invoiceId", how="left")

##################################################

#Final claim and Provider df
print(df_claims_provider_details.info())
print(df_claims_provider_details)

'''

'\n### All Claim details (took 2 minutes to execute)\n\ndf_all_claims = pd.read_sql("select c.id as claim_id, invoice_id as invoiceId, c.item_category_level3_id from HH_claim c", con=conn)\n\n#print(df_all_claims.info())\n#print(df_all_claims)\n### All item_categories details. Took few seconds for execution.\n\n##  \ndf_all_ndis_service_cat = pd.read_sql("select item_category_level3_id, ndis.registration_group from hedgehog_ndis_service_item_ref ndis", con=conn)\n#print(df_all_ndis_service_cat.info())\n#print(df_all_ndis_service_cat)\n\n##Combine df_all_claims and df_all_ndis_service_cat\ndf_claimsWithProvider_details = pd.merge(df_all_claims, df_all_ndis_service_cat, on="item_category_level3_id", how= \'inner\')\ndf_claimsWithProvider_details= df_claimsWithProvider_details.drop(columns=[\'item_category_level3_id\'])\n#df_claimsWithProvider_details = pd.read_sql("select c.id, c.invoice_id as invoiceId, n.registration_group from HH_claim c left join hedgehog_ndis_service_item_ref n on c